In [9]:
from pathlib import Path

from typing import Any, Literal
from collections.abc import Iterable
from typing_extensions import Self

from numpy.typing import NDArray
from pandas import DataFrame

import random
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neighbors import KernelDensity

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm

from cpscheduler.environment import SchedulingEnv, SingleMachineSetup, WeightedTardiness, Objective

from cpscheduler.solver import PulpSolver
from cpscheduler.heuristics import (
    PriorityDispatchingRule,
    WeightedShortestProcessingTime,
    ModifiedDueDate,
    ApparentTardinessCost,
    TrafficPriority,
    CostOverTime,
)

KERNEL = Literal['gaussian', 'tophat', 'epanechnikov', 'exponential', 'linear', 'cosine']

INSTANCE_PATH = Path("data/customers/wt40_50_0.5.pkl")

kernel: KERNEL = 'gaussian'
n_instances: int = 500
bandwidth: float = 10.
n_jobs: int = 40

solver_tag: str = "CPLEX_CMD"
timelimit: int = 30

pdrs: dict[str, PriorityDispatchingRule] = {
    "WSPT": WeightedShortestProcessingTime(strict=True),
    "MDD" : ModifiedDueDate(strict=True),
    "WMDD": ModifiedDueDate(weight_label="weight", strict=True),
    "COverT": CostOverTime(strict=True),
    "ATC": ApparentTardinessCost(5, strict=True),
    "TP" : TrafficPriority(strict=True),
}

choices = [*pdrs.keys(), "Optimal"]

In [2]:
list_instances: list[DataFrame]
instance_info: DataFrame
customer_information: DataFrame

list_instances, instance_info, customer_information = pd.read_pickle(INSTANCE_PATH)

In [3]:
class GroupedKDEGenerator(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        group_column: str = 'customer',
        column_names: Iterable[str] | None = None,
        kernel: KERNEL = "gaussian",
        bandwidth: float = 0.1,
        **kwargs: Any,
    ):
        super().__init__()

        self.group_column = group_column
        self.kernel = kernel
        self.bandwidth = bandwidth
        self.kwargs = kwargs

        self.kde_models: dict[str, KernelDensity] = {}
        self.group_frequencies: dict[str, float] = {}
        self.column_names: list[str] = list(column_names) if column_names is not None else []

    def fit(self, X: DataFrame, y: NDArray[np.floating[Any]] | None = None) -> Self:
        if not self.column_names:
            self.column_names = [col for col in X.columns if col != self.group_column]

        else:
            X = X[self.column_names + [self.group_column]]

        self.kde_models.clear()
        for group, group_data in X.groupby(self.group_column):
            kde = KernelDensity(kernel=self.kernel, bandwidth=self.bandwidth, **self.kwargs)
            kde.fit(group_data.drop(columns=[self.group_column]))
            self.kde_models[str(group)] = kde
            self.group_frequencies[str(group)] = len(group_data) / len(X)

        self.is_fitted_ = True
        return self

    def sample(self, n_samples: int, random_state: np.random.Generator | None = None) -> DataFrame:
        if not self.is_fitted_:
            raise RuntimeError("The model must be fitted before sampling.")

        if random_state is None:
            random_state = np.random.default_rng()

        group_sample = random_state.multinomial(n_samples, list(self.group_frequencies.values()))

        samples: list[NDArray[Any]] = []
        for group, count in zip(self.group_frequencies, group_sample):
            if count > 0:
                kde = self.kde_models[group]
                group_samples = kde.sample(count)
                samples.append(group_samples)

        all_samples = np.vstack(samples)

        df =  DataFrame(all_samples, columns=self.column_names)
        df[self.group_column] = np.concatenate([[group] * count for group, count in zip(self.group_frequencies, group_sample)])

        return df


In [4]:
kde_generator = GroupedKDEGenerator(
    group_column="customer",
    column_names=["processing_time", "due_date"],
    kernel=kernel,
    bandwidth=bandwidth,
)
kde_generator.fit(pd.concat(list_instances))

rng = random.Random(0)
np_rng = np.random.default_rng(0)
training_instances: list[DataFrame] = []

env = SchedulingEnv(
    SingleMachineSetup(),
    objective=WeightedTardiness(),
)

with tqdm(range(n_instances), unit="instance",) as pbar:
    pbar.set_description("Generating training instances with behavior data")
    pbar.set_postfix_str("")

    for i, instance in enumerate(pbar):
        instance = kde_generator.sample(
            n_samples=n_jobs,
            random_state=np_rng,
        )

        instance['customer'] = instance['customer'].astype(int)
        instance['processing_time'] = instance['processing_time'].astype(int).clip(1, 100)
        instance['due_date'] = instance['due_date'].astype(int).clip(0)
        instance['weight'] = customer_information["weight"].loc[instance['customer']].to_numpy()

        env.set_instance(instance)
        obs, info = env.reset()

        strategy = rng.choice(choices)
        pbar.set_postfix_str(f"Strategy: {strategy}")

        if strategy == "Optimal":
            solver = PulpSolver(env)
            action, _, _ = solver.solve(solver_tag, quiet=True, time_limit=timelimit)

        else:
            pdr = pdrs[strategy]
            action = pdr.sample(obs, info["current_time"], target_prob=0.9, seed=i)

        env.step(action)

        instance["BPolicy start"] = [task.get_start() for task in env.tasks]

        training_instances.append(instance.drop(columns=["weight"]))


Generating training instances with behavior data:   0%|                                     | 0/500 [00:00<?, ?instance/s, Strategy: Optimal]

Generating training instances with behavior data: 100%|████████████████████████████| 500/500 [36:53<00:00,  4.43s/instance, Strategy: COverT]


In [ ]:
pd.to_pickle(
    training_instances,
    INSTANCE_PATH.parent / f"{INSTANCE_PATH.stem}_dataset.pkl"
)